# Convert foreign payments to USD with historical FX rates

For each type of payment that needs to be converted to USD, and instance of ```FatcatCalculator``` is defined, which is instantiated with the currency of the payments, and a list of dicts containing the payment dates and amounts.

In [1]:
import pandas as pd
from datetime import datetime

from expat_fatcat import FatcatCalculator, QuandlUSDRateConverterTo, f2555
from expat_fatcat.expat_fatcat import DummyRateConverterTo

# Load example payment data
This data is represents fake payment data in the currency "FOO".

In [2]:
payments = pd.read_csv('example.csv')
payments

,date,amount
0,2017-01-20,1000
1,2017-02-20,1000
2,2017-03-20,1500


In [3]:
converter = DummyRateConverterTo('USD')
payment_calculator = FatcatCalculator(converter)
print('Example aggregate payment calculation: {}'
      .format(payment_calculator('FOO', payments.to_dict(orient='records'))))

Example aggregate payment calculation: 3937.5


## Compute total foreign income  and housing deduction examples of f2555

In [4]:
salary = pd.read_csv('example_salary.csv').to_dict(orient='records')
calculator = FatcatCalculator(converter)
f2555_line_25 = calculator('FOO', salary)
print('Example salary calculation, line 25 of f2555: {}'.format(f2555_line_25))

housing = pd.read_csv('example_housing.csv').to_dict(orient='records')
f2555_line_34 = calculator('FOO', housing)
print('Example housing deduction calculation, line 34 of f2555: {}'.format(f2555_line_34))

Example salary calculation, line 25 of f2555: 3937.5
Example housing deduction calculation, line 34 of f2555: 2193.75


## Putting things together with the function ```f2555```

In [5]:
converter = DummyRateConverterTo('USD')
data = [
    {'tag': 'salary', 'currency': 'FOO', 'payments': salary},
    {'tag': 'rent', 'currency': 'BAR', 'payments': housing}
]
f2555(converter, data)

{'salary': {'currency': 'FOO', 'amount': 3937.5},
 'rent': {'currency': 'BAR', 'amount': 2193.75}}

## Using QUANDL to get fx rates
If you have a quandl api key, you can get real exchange rates using the ```QuandlUSDRateConverter```. For the below examples, we pretend our payments are now in EUR.

In [6]:
converter = QuandlUSDRateConverterTo()
payment_calculator = FatcatCalculator(converter)
print('Example payment EUR to USD calculation: {}'
      .format(payment_calculator('EUR', payments.to_dict(orient='records'))))

Example payment EUR to USD calculation: 3737.6000000000004


### Imputing missing rates

In [7]:
# Bank holiday: Good Friday 2019
converter.get_rate('EUR', datetime(2019, 4, 19))

Invalid date, trying before and after
Invalid date, trying before and after
Invalid date, trying before and after


1.125

In [8]:
# Good Friday rate is taken from the day before
converter.get_rate('EUR', datetime(2019, 4, 19)) == converter.get_rate('EUR', datetime(2019, 4, 18))

Invalid date, trying before and after
Invalid date, trying before and after
Invalid date, trying before and after


True

In [9]:
# Easter Sunday rate is taken from following Tuesday
converter.get_rate('EUR', datetime(2019, 4, 21)) == converter.get_rate('EUR', datetime(2019, 4, 23))

Invalid date, trying before and after
Invalid date, trying before and after
Invalid date, trying before and after
Invalid date, trying before and after
Invalid date, trying before and after


True